In [118]:
from pathlib import Path
import pickle
import re

from tqdm import tqdm, tqdm_notebook

import numpy as np
import pandas as pd
from scipy import sparse

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.preprocessing import FunctionTransformer, LabelEncoder

from sklearn.linear_model import LogisticRegression, PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.svm import LinearSVC

In [119]:
pd.options.display.max_colwidth = 50 # default - 50
tqdm_notebook().pandas()

In [120]:
DATA_PATH = Path('../data')
RANDOM_SEED = 17

**Data Loading**

In [121]:
train_df = pd.read_csv(DATA_PATH/'train.csv')
valid_df = pd.read_csv(DATA_PATH/'valid.csv')
test_df = pd.read_csv(DATA_PATH/'test.csv')

In [122]:
with open(DATA_PATH/'X_train_ftfy_spacy.pkl', 'rb') as  f:
    X_train_clean = pickle.load(f)

train_df['title'] = X_train_clean['clean_title']
train_df['text'] = X_train_clean['clean_text']

with open(DATA_PATH/'X_valid_ftfy_spacy.pkl', 'rb') as  f:
    X_valid_clean = pickle.load(f)

valid_df['title'] = X_valid_clean['clean_title']
valid_df['text'] = X_valid_clean['clean_text']

with open(DATA_PATH/'X_test_ftfy_spacy.pkl', 'rb') as  f:
    X_test_clean = pickle.load(f)

test_df['title'] = X_test_clean['clean_title']
test_df['text'] = X_test_clean['clean_text']

**Data Cleaning**

In [123]:
X_train = train_df.fillna('')
X_valid = valid_df.fillna('')
X_test = test_df.fillna('')

In [124]:
le = LabelEncoder()
X_train['class'] = le.fit_transform(X_train['label'])
X_valid['class'] = le.transform(X_valid['label'])

In [125]:
le.classes_

array(['clickbait', 'news', 'other'], dtype=object)

In [126]:
X_train_pos_title = X_train['title'].str.findall(r'_([A-Z]+)').progress_apply(lambda x: ' '.join(x))
X_train_pos_text = X_train['text'].str.findall(r'_([A-Z]+)').progress_apply(lambda x: ' '.join(x))
X_train_pos_title.head(), X_train_pos_text.head()

(0    PROPN CCONJ PROPN PROPN PUNCT PROPN PROPN PART...
 1    NOUN PART VERB DET PROPN NOUN ADP PROPN PROPN ...
 2    DET PROPN PROPN ADP DET PROPN PUNCT VERB ADP P...
 3    PROPN PART PROPN VERB PROPN PROPN ADP PROPN PR...
 4    NOUN VERB PRON VERB NOUN NOUN PART NOUN ADP DE...
 Name: title, dtype: object,
 0    NOUN ADV VERB PUNCT PROPN VERB VERB ADJ ADJ CC...
 1    PROPN NUM ADJ PROPN PROPN PROPN PROPN VERB PRO...
 2    NOUN VERB ADP DET NOUN ADP NOUN CCONJ NOUN ADP...
 3    DET NOUN ADP NOUN VERB ADP PROPN PROPN PUNCT D...
 4    DET ADJ NOUN NOUN VERB VERB ADP NOUN ADP PRON ...
 Name: text, dtype: object)

In [127]:
X_valid_pos_title = X_valid['title'].str.findall(r'_([A-Z]+)').progress_apply(lambda x: ' '.join(x))
X_valid_pos_text = X_valid['text'].str.findall(r'_([A-Z]+)').progress_apply(lambda x: ' '.join(x))
X_valid_pos_title.head(), X_valid_pos_text.head()

(0    PROPN VERB PRON VERB VERB NOUN PART ADJ ADP NO...
 1    PROPN PROPN PART NOUN VERB ADJ ADJ NOUN ADP PROPN
 2    PROPN PROPN VERB SYM NUM NUM ADP PROPN PROPN P...
 3    NOUN VERB ADJ ADP DET PROPN PROPN PROPN PROPN ...
 4    PROPN PROPN CCONJ PROPN PROPN PROPN PROPN PART...
 Name: title, dtype: object,
 0    PROPN PROPN PUNCT PROPN PROPN NUM PUNCT NUM NU...
 1    PROPN VERB VERB VERB DET NOUN ADP PROPN ADP PR...
 2    PROPN PUNCT DET PROPN NOUN VERB PROPN PRON VER...
 3    DET ADJ NOUN NOUN VERB NOUN NOUN NOUN VERB VER...
 4    DET NOUN ADP PROPN PROPN VERB ADJ PROPN NOUN P...
 Name: text, dtype: object)

In [128]:
X_test_pos_title = X_test['title'].str.findall(r'_([A-Z]+)').progress_apply(lambda x: ' '.join(x))
X_test_pos_text = X_test['text'].str.findall(r'_([A-Z]+)').progress_apply(lambda x: ' '.join(x))
X_test_pos_title.head(), X_test_pos_text.head()

(0    PROPN PROPN PROPN PROPN VERB ADV DET ADJ ADJ N...
 1    VERB PROPN PROPN PROPN DET PROPN ADP DET PROPN...
 2    ADP DET NOUN PART NOUN NOUN PUNCT VERB ADP ADP...
 3    NUM PROPN PART VERB ADJ NOUN PART VERB PART AD...
 4    PROPN VERB VERB PRON VERB VERB ADP PROPN VERB ...
 Name: title, dtype: object,
 0    ADJ VERB PROPN PROPN ADP PROPN SPACE PROPN PRO...
 1    ADJ PROPN PROPN VERB PART VERB ADV DET NOUN PU...
 2    PROPN VERB DET NOUN ADP ADJ PROPN ADP DET ADJ ...
 3    NOUN VERB ADP NOUN VERB DET NOUN ADP DET ADJ N...
 4    VERB DET ADP PROPN PROPN PROPN PROPN PROPN PRO...
 Name: text, dtype: object)

In [129]:
X_train['title_pos'] = X_train_pos_title
X_train['text_pos'] = X_train_pos_text

X_valid['title_pos'] = X_valid_pos_title
X_valid['text_pos'] = X_valid_pos_text

X_test['title_pos'] = X_test_pos_title
X_test['text_pos'] = X_test_pos_text

**Training**

In [130]:
class ColumnExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, columns=None):
        self.columns = columns
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return X[self.columns]

In [131]:
class NBTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, alpha=1.0):
        self.alpha = alpha

    def fit(self, X, y=None):
        y = y.values
        
        pos_count = X[y==1].sum(0) 
        neg_count = X[y==0].sum(0)
        n = X.shape[1]
        p = (pos_count + self.alpha) / (pos_count.sum() + self.alpha * n)
        q = (neg_count + self.alpha) / (neg_count.sum() + self.alpha * n)
        self.r_ = np.log(p / q)
        return self
    
    def transform(self, X, y=None):
        return X.multiply(self.r_)

In [132]:
class TfidfVectorizerPlus(TfidfVectorizer):
    def __init__(self, fit_add=None, norm_type=None, pivot=5, slope=0.2, 
                       input='content', encoding='utf-8', decode_error='strict', 
                       strip_accents=None, lowercase=True, preprocessor=None, 
                       tokenizer=None, analyzer='word', stop_words=None, 
                       token_pattern='(?u)\\b\\w\\w+\\b', ngram_range=(1, 1), 
                       max_df=1.0, min_df=1, max_features=None, vocabulary=None, 
                       binary=False, dtype=np.float64, norm='l2', 
                       use_idf=True, smooth_idf=True, sublinear_tf=False):
        super().__init__(input, encoding, decode_error,
                         strip_accents, lowercase, preprocessor,
                         tokenizer, analyzer, stop_words,
                         token_pattern, ngram_range,
                         max_df, min_df, max_features, vocabulary,
                         binary, dtype, norm,
                         use_idf, smooth_idf, sublinear_tf)
        
        self.fit_add = fit_add
        self.norm_type = norm_type
        self.pivot = pivot
        self.slope = slope
    
    def fit(self, X, y=None):
        if self.fit_add is not None:
            X_new = pd.concat([X, self.fit_add])
        else:
            X_new = X
        
        super().fit(X_new, y)
        return self
        
    def transform(self, X, y=None):
        res = super().transform(X)
            
        if self.norm_type == 'pivot_cosine':
            norm_factor = (1 - self.slope) * self.pivot + self.slope * sparse.linalg.norm(res, axis=1).reshape(-1, 1)
            res = sparse.csr_matrix(res.multiply(1 / norm_factor))
        elif self.norm_type == 'pivot_unique':
            unique_terms_num = (res > 0).sum(axis=1)
            norm_factor = (1 - self.slope) * self.pivot + self.slope * unique_terms_num
            res = sparse.csr_matrix(res.multiply(1 / norm_factor))
        elif self.norm_type is not None:
            raise ValueError('Incorrect normalization type')
            
        return res

In [133]:
class TextTruncater(BaseEstimator, TransformerMixin):
    def __init__(self, max_length=None):
        self.max_length = max_length
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        if self.max_length is not None:
            return X.str[:self.max_length]
        else:
            return X

In [135]:
class ModelTransformer(TransformerMixin):

    def __init__(self, model):
        self.model = model

    def fit(self, *args, **kwargs):
        self.model.fit(*args, **kwargs)
        return self

    def transform(self, X, **transform_params):
        return DataFrame(self.model.predict_proba(X))

In [164]:
pipe = Pipeline([
    ('features', FeatureUnion([
        ('title', Pipeline([
            ('extract', ColumnExtractor(columns='title')),
            ('vec', TfidfVectorizer())
        ])),
        ('text', Pipeline([
            ('extract', ColumnExtractor(columns='text')),
            ('vec', TfidfVectorizer())
        ])),
#         ('title_pos', Pipeline([
#             ('extract', ColumnExtractor('title_pos')),
#             ('vec', TfidfVectorizer())
#         ])),
        ('text_pos', Pipeline([
            ('extract', ColumnExtractor('text_pos')),
            ('vec', TfidfVectorizer())
        ])),
    ], 
#         transformer_weights={
#             'title': 0.4,
#             'text': 0.6,
#         }
    )),
    ('clf', LogisticRegression())
])

In [165]:
param_grid = {
    'features__title__vec': [TfidfVectorizer()],
    'features__title__vec__strip_accents': ['unicode'], #[None, 'unicode', 'ascii'],
    'features__title__vec__lowercase': [False], #[True, False],
    'features__title__vec__analyzer': ['word'], #['word', 'char', 'char_wb'],
    'features__title__vec__stop_words': ['english'], #[None, 'english'],
    'features__title__vec__token_pattern': [r'\b\w{5,}\b'], #, r'(?u)\b\w\w+\b'], #['r'\b\w+\b',']
    'features__title__vec__ngram_range': [(1, 3)], #[(1, 3), (1, 4), (1, 5)],
    'features__title__vec__max_df': [0.5],
    'features__title__vec__min_df': [1],
    'features__title__vec__max_features': [None], #[None, 50000, 100000],
    'features__title__vec__binary': [True], #[True, False],
    'features__title__vec__use_idf': [True], #[True, False],
    'features__title__vec__smooth_idf': [True], #[True, False],
    'features__title__vec__sublinear_tf': [True], #[True, False],


    'features__text__vec': [TfidfVectorizer()],
    'features__text__vec__strip_accents': ['ascii'], #[None, 'unicode', 'ascii'],
    'features__text__vec__lowercase': [False], #[True, False],
    'features__text__vec__analyzer': ['word'], #['word', 'char', 'char_wb'],
    'features__text__vec__stop_words': [None], #[None, 'english'],
    #               'features__text__vec__token_pattern': [r'\b\w+\b'], #[r'\b\w+\b', r'(?u)\b\w\w+\b', r'\b\w{3,}\b', r'\b\w{4,}\b'],
    'features__text__vec__ngram_range': [(1, 2)],
    'features__text__vec__max_df': [0.4],
    'features__text__vec__min_df': [2],
    'features__text__vec__max_features': [150000],
    'features__text__vec__binary': [True], #[True, False],
    'features__text__vec__use_idf': [True], #[True, False],
    'features__text__vec__smooth_idf': [False], #[True, False],
    'features__text__vec__sublinear_tf': [True], #[True, False],
     
#     'features__title_pos__vec': [TfidfVectorizer()],
#     'features__title_pos__vec__strip_accents': [None], #[None, 'unicode', 'ascii'],
#     'features__title_pos__vec__lowercase': [True], #[True, False],
#     'features__title_pos__vec__analyzer': ['word'], #['word', 'char', 'char_wb'],
#     'features__title_pos__vec__stop_words': [None], #[None, 'english'],
#     'features__title_pos__vec__ngram_range': [(1, 6)],
#     'features__title_pos__vec__max_df': [0.4],
#     'features__title_pos__vec__min_df': [60],
#     'features__title_pos__vec__max_features': [None], #[None, 50000, 100000],
#     'features__title_pos__vec__binary': [False], #[True, False],
#     'features__title_pos__vec__use_idf': [True], #[True, False],
#     'features__title_pos__vec__smooth_idf': [True], #[True, False],
#     'features__title_pos__vec__sublinear_tf': [True], #[True, False],
    
    'features__text_pos__vec': [TfidfVectorizer()],
    'features__text_pos__vec__strip_accents': [None], #[None, 'unicode', 'ascii'],
    'features__text_pos__vec__lowercase': [True], #[True, False],
    'features__text_pos__vec__analyzer': ['word'], #['char', 'char_wb'],
    'features__text_pos__vec__stop_words': [None], #[None, 'english'],
    'features__text_pos__vec__ngram_range': [(1, 3)],
    'features__text_pos__vec__max_df': [1.0],
    'features__text_pos__vec__min_df': [10],
    'features__text_pos__vec__max_features': [None], #[None, 50000, 100000],
    'features__text_pos__vec__binary': [True], #[True, False],
    'features__text_pos__vec__use_idf': [False], #[True, False],
    'features__text_pos__vec__smooth_idf': [True], #[True, False],
    'features__text_pos__vec__sublinear_tf': [True], #[True, False],


    'clf': [LogisticRegression()],
    'clf__penalty': ['l2'], #['l1', 'l2'], # ['l2'],
    'clf__C': [1], #np.logspace(-2, 2, 5), # [2], 
    'clf__class_weight': ['balanced'], #['balanced']
    'clf__random_state': [RANDOM_SEED],
    'clf__solver':  ['lbfgs'], #['lbfgs']
    'clf__max_iter': [200],
    'clf__multi_class': ['multinomial'], #['ovr', 'multinomial'],
              
              
#               'features__title__vec': [TfidfVectorizer()],
#               'features__title__vec__strip_accents': [None, 'unicode', 'ascii'],
#               'features__title__vec__lowercase': [True, False],
#               'features__title__vec__analyzer': ['word', 'char', 'char_wb'],
#               'features__title__vec__stop_words': [None, 'english'],
#               'features__title__vec__token_pattern': [r'\b\w+\b'], #[r'\b\w+\b', r'(?u)\b\w\w+\b'],
#               'features__title__vec__ngram_range': [(1, 4)],
#               'features__title__vec__max_df': [0.8],
#               'features__title__vec__min_df': [1], #[1, 5, 10],
#               'features__title__vec__max_features': [70000],
#               'features__title__vec__binary': [True], #[True, False],
#               'features__title__vec__use_idf': [True], #[True, False],
#               'features__title__vec__smooth_idf': [True], #[True, False],
#               'features__title__vec__sublinear_tf': [True], #[True, False],

#               'clf': [LinearSVC()],
#               'clf__penalty': ['l2'],
#               'clf__loss': ['squared_hinge'], #['squared_hinge', 'hinge'],
#               'clf__dual': [False], #[True, False],
#               'clf__C': [0.4],
#               'clf__class_weight': ['balanced'],
#               'clf__random_state': [random_seed],
              
#               'features__text__vec': [TfidfVectorizer()],
#               'features__text__vec__strip_accents': ['ascii'], #[None, 'unicode', 'ascii'],
#               'features__text__vec__lowercase': [False], #[True, False],
#               'features__text__vec__analyzer': ['word'], #['word', 'char', 'char_wb'],
#               'features__text__vec__stop_words': [None], #[None, 'english'],
#               'features__text__vec__token_pattern': [r'\b\w+\b'], #[r'\b\w+\b', r'(?u)\b\w\w+\b'],
#               'features__text__vec__ngram_range': [(1, 1), (1, 2), (1, 3)],
#               'features__text__vec__max_df': [0.9],
#               'features__text__vec__min_df': [1],
#               'features__text__vec__max_features': [200000, 300000, 400000, 500000],
#               'features__text__vec__binary': [False], #[True, False],
#               'features__text__vec__use_idf': [True], #[True, False],
#               'features__text__vec__smooth_idf': [False], #[True, False],
#               'features__text__vec__sublinear_tf': [True], #[True, False],
              
#               'features__text__nb_features__alpha': np.linspace(0.1, 1, 10),
    
#               'clf': [LinearSVC()],
# #               'clf__penalty': ['l2'],
# #               'clf__loss': ['squared_hinge', 'hinge'],
# #               'clf__dual': [False], #[True, False],
#               'clf__C': [1],
#               'clf__multi_class': ['crammer_singer'], #['ovr', 'crammer_singer']
#               'clf__class_weight': ['balanced'], #[None, 'balanced'],
#               'clf__random_state': [RANDOM_SEED],
                 
}

cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=RANDOM_SEED)

In [166]:
grid_search = GridSearchCV(pipe, param_grid, scoring='f1_macro', 
                           cv=cv, n_jobs=-1, return_train_score=False,
                           verbose=2, iid=True)

grid_search.fit(X_train, X_train['class'])

Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  2.0min finished


GridSearchCV(cv=StratifiedKFold(n_splits=2, random_state=17, shuffle=True),
       error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=None,
       transformer_list=[('title', Pipeline(memory=None,
     steps=[('extract', ColumnExtractor(columns='title')), ('vec', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', inp...penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'features__title__vec': [TfidfVectorizer(analyzer='word', binary=True, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=0.5, max_features=None, min_df=1,
        ngram_range=(1, 3), norm='l2', preprocessor=No...state': [17], 'clf__solver': ['lbfgs'], 'clf__max_iter': [200], 'clf__multi_cla

0.8058100230051004

In [167]:
grid_search.best_score_, grid_search.best_params_

(0.8018456296764946,
 {'clf': LogisticRegression(C=1, class_weight='balanced', dual=False,
            fit_intercept=True, intercept_scaling=1, max_iter=200,
            multi_class='multinomial', n_jobs=None, penalty='l2',
            random_state=17, solver='lbfgs', tol=0.0001, verbose=0,
            warm_start=False),
  'clf__C': 1,
  'clf__class_weight': 'balanced',
  'clf__max_iter': 200,
  'clf__multi_class': 'multinomial',
  'clf__penalty': 'l2',
  'clf__random_state': 17,
  'clf__solver': 'lbfgs',
  'features__text__vec': TfidfVectorizer(analyzer='word', binary=True, decode_error='strict',
          dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
          lowercase=False, max_df=0.4, max_features=150000, min_df=2,
          ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=False,
          stop_words=None, strip_accents='ascii', sublinear_tf=True,
          token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
          vocabulary=None),

In [168]:
cv_results_df = pd.DataFrame(grid_search.cv_results_).T
cv_results_df

,0
mean_fit_time,84.8705
std_fit_time,1.59469
mean_score_time,27.6077
std_score_time,0.2943
param_clf,"LogisticRegression(C=1, class_weight='balanced..."
param_clf__C,1
param_clf__class_weight,balanced
param_clf__max_iter,200
param_clf__multi_class,multinomial
param_clf__penalty,l2


In [169]:
y_val_pred = grid_search.predict_proba(X_valid).argmax(axis=1)
y_val_pred

array([1, 1, 1, ..., 2, 1, 1])

In [170]:
le.inverse_transform(y_val_pred)

array(['news', 'news', 'news', ..., 'other', 'news', 'news'], dtype=object)

In [171]:
np.bincount(y_val_pred) / len(y_val_pred)

array([0.15990991, 0.57516892, 0.26492117])

**Evaluation**

In [172]:
f1_score(X_valid['class'], y_val_pred, average='macro')

0.80614719136929

**Feature Importance**

**Predict & Submit**

In [173]:
full_train_df = pd.concat([X_train, X_valid], axis=0, ignore_index=True)

In [174]:
best_model = grid_search.best_estimator_
best_model.fit(full_train_df, full_train_df['class'])

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=None,
       transformer_list=[('title', Pipeline(memory=None,
     steps=[('extract', ColumnExtractor(columns='title')), ('vec', TfidfVectorizer(analyzer='word', binary=True, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', inpu...l2',
          random_state=17, solver='lbfgs', tol=0.0001, verbose=0,
          warm_start=False))])

In [175]:
X_test.columns

Index(['id', 'title', 'text', 'title_pos', 'text_pos'], dtype='object')

In [176]:
X_test.columns = ['id', 'title', 'text', 'title_pos', 'text_pos']#, 'title_length', 'text_length',
       #'is_title_na', 'is_text_na']
X_test.head()

,id,title,text,title_pos,text_pos
0,0,Amazon_PROPN CEO_PROPN Jeff_PROPN Bezos_PROPN ...,More_ADJ Try_VERB Yahoo_PROPN Finance_PROPN on...,PROPN PROPN PROPN PROPN VERB ADV DET ADJ ADJ N...,ADJ VERB PROPN PROPN ADP PROPN SPACE PROPN PRO...
1,1,Does_VERB Laura_PROPN Dern_PROPN Handle_PROPN ...,More_ADJ Laura_PROPN Dern_PROPN seems_VERB to_...,VERB PROPN PROPN PROPN DET PROPN ADP DET PROPN...,ADJ PROPN PROPN VERB PART VERB ADV DET NOUN PU...
2,2,In_ADP this_DET photographer_NOUN 's_PART home...,Kirkuk_PROPN is_VERB a_DET city_NOUN of_ADP No...,ADP DET NOUN PART NOUN NOUN PUNCT VERB ADP ADP...,PROPN VERB DET NOUN ADP ADJ PROPN ADP DET ADJ ...
3,3,8_NUM Ways_PROPN To_PART Get_VERB Your_ADJ Spo...,Experts_NOUN say_VERB that_ADP communication_N...,NUM PROPN PART VERB ADJ NOUN PART VERB PART AD...,NOUN VERB ADP NOUN VERB DET NOUN ADP DET ADJ N...
4,4,US_PROPN says_VERB claim_VERB it_PRON supporte...,Share_VERB this_DET with_ADP Email_PROPN Faceb...,PROPN VERB VERB PRON VERB VERB ADP PROPN VERB ...,VERB DET ADP PROPN PROPN PROPN PROPN PROPN PRO...


In [177]:
(test_df.index == X_test.id).all()

True

In [178]:
y_test_pred = best_model.predict_proba(X_test).argmax(axis=1)
y_test_pred

array([0, 0, 1, ..., 1, 1, 1])

In [179]:
y_test_label = le.inverse_transform(y_test_pred)
y_test_label

array(['clickbait', 'clickbait', 'news', ..., 'news', 'news', 'news'],
      dtype=object)

In [180]:
np.bincount(y_test_pred) / len(y_test_label)

array([0.2066584 , 0.7060386 , 0.08730299])

In [181]:
submission_df = pd.DataFrame({'id': X_test['id'], 'label': y_test_label})
submission_df.head()

,id,label
0,0,clickbait
1,1,clickbait
2,2,news
3,3,clickbait
4,4,news


In [182]:
submission_df.to_csv('submission.csv', index=False)

In [183]:
!head submission.csv

id,label
0,clickbait
1,clickbait
2,news
3,clickbait
4,news
5,news
6,news
7,news
8,news


In [184]:
!wc -l submission.csv

5648 submission.csv


In [185]:
!kaggle competitions submit -c dlinnlp-spring-2019-clf -f submission.csv -m "LR (mn)"

100%|██████████████████████████████████████| 60.3k/60.3k [00:03<00:00, 16.0kB/s]
Successfully submitted to DL in NLP Spring 2019. Classification

**Error Analysis**

In [215]:
y_margins = grid_search.decision_function(X_valid)
y_val_pred = (y_margins - y_margins.min()) / (y_margins.max() - y_margins.min())
# y_val_pred = y_val_pred.argmax(axis=1)
y_val_pred

array([[0.59144613, 0.77370618, 0.36949007],
       [0.4860577 , 0.67253872, 0.57604596],
       [0.42553417, 0.79839178, 0.51071644],
       ...,
       [0.44698388, 0.51817221, 0.7694863 ],
       [0.66384886, 0.84248302, 0.22831051],
       [0.58173126, 0.84916822, 0.30374291]])

In [220]:
validate_df = X_valid.copy()
validate_df = validate_df.assign(pred_0 = y_val_pred[:, 0], 
                                 pred_1 = y_val_pred[:, 1],
                                 pred_2 = y_val_pred[:, 2],
                                 class_pred = y_val_pred.argmax(axis=1))
validate_df.head()

,label,title,text,class,pred_0,pred_1,pred_2,class_pred
0,news,Trump_PROPN says_VERB he_PRON is_VERB releasin...,"Bob_PROPN Bryan_PROPN ,_PUNCT Business_PROPN I...",1,0.591446,0.773706,0.369490,1
1,news,Fidel_PROPN Castro_PROPN 's_PART ashes_NOUN ma...,Cubans_PROPN have_VERB been_VERB lining_VERB t...,1,0.486058,0.672539,0.576046,1
2,news,Obama_PROPN Administration_PROPN Sending_VERB ...,WASHINGTON_PROPN —_PUNCT The_DET Obama_PROPN a...,1,0.425534,0.798392,0.510716,1
3,news,Insurers_NOUN Are_VERB Worried_ADJ About_ADP T...,The_DET main_ADJ industry_NOUN groups_NOUN rep...,1,0.575398,0.764276,0.394968,1
4,news,Kobe_PROPN Bryant_PROPN and_CCONJ Nike_PROPN F...,A_DET year_NOUN after_ADP Kobe_PROPN Bryant_PR...,1,0.543219,0.730411,0.461013,1


In [221]:
incorrect_df = validate_df[validate_df['class'] != validate_df['class_pred']]
incorrect_df.head()

,label,title,text,class,pred_0,pred_1,pred_2,class_pred
123,news,How_ADV to_PART Talk_VERB to_ADP Your_ADJ Teen...,"More_ADJ For_ADP parents_NOUN ,_PUNCT talking_...",1,0.746912,0.720410,0.267320,0
229,news,Genius_PROPN Bar_PROPN secrets_NOUN revealed_V...,One_NUM of_ADP the_DET biggest_ADJ things_NOUN...,1,0.598378,0.581579,0.554686,0
347,news,13_NUM Gifts_NOUN For_ADP People_NOUN Who_NOUN...,Love_NOUN is_VERB n't_ADV all_DET we_PRON need...,1,0.635953,0.620099,0.478591,0
391,news,Deer_PROPN Debacle_NOUN Exacerbates_VERB New_P...,Desc_PROPN,1,0.416454,0.626502,0.691686,2
490,news,Donald_PROPN Trump_PROPN Pays_VERB Respects_NO...,President_PROPN Donald_PROPN Trump_PROPN trave...,1,0.712625,0.705318,0.316700,0


In [238]:
validate_df.groupby(['class', 'class_pred'])['label'].count()

class  class_pred
0      0              542
       1                1
1      0               19
       1             2065
       2                4
2      2              921
Name: label, dtype: int64

In [239]:
incorrect_df.groupby(['class', 'class_pred'])['label'].count()

class  class_pred
0      1              1
1      0             19
       2              4
Name: label, dtype: int64

In [240]:
incorrect_df

,label,title,text,class,pred_0,pred_1,pred_2,class_pred
123,news,How_ADV to_PART Talk_VERB to_ADP Your_ADJ Teen...,"More_ADJ For_ADP parents_NOUN ,_PUNCT talking_...",1,0.746912,0.720410,0.267320,0
229,news,Genius_PROPN Bar_PROPN secrets_NOUN revealed_V...,One_NUM of_ADP the_DET biggest_ADJ things_NOUN...,1,0.598378,0.581579,0.554686,0
347,news,13_NUM Gifts_NOUN For_ADP People_NOUN Who_NOUN...,Love_NOUN is_VERB n't_ADV all_DET we_PRON need...,1,0.635953,0.620099,0.478591,0
391,news,Deer_PROPN Debacle_NOUN Exacerbates_VERB New_P...,Desc_PROPN,1,0.416454,0.626502,0.691686,2
490,news,Donald_PROPN Trump_PROPN Pays_VERB Respects_NO...,President_PROPN Donald_PROPN Trump_PROPN trave...,1,0.712625,0.705318,0.316700,0
497,news,These_DET trapdoor_NOUN waterslides_NOUN plung...,"David_PROPN Ibekwe_PROPN Apr._PROPN 12_NUM ,_P...",1,0.634602,0.630173,0.469868,0
694,news,How_ADV '_PUNCT Jurassic_PROPN World_PROPN '_P...,It_PRON is_VERB well_ADV -_PUNCT known_VERB at...,1,0.720751,0.715298,0.298594,0
726,news,10_NUM things_NOUN you_PRON need_VERB to_PART ...,"Jonathan_PROPN Garber_PROPN ,_PUNCT Business_P...",1,0.759789,0.700965,0.273889,0
754,news,Regis_PROPN Philbin_PROPN :_PUNCT Trump_PROPN ...,Daytime_ADJ talk_NOUN show_NOUN legend_NOUN Re...,1,0.780099,0.662219,0.292325,0
991,news,What_NOUN will_VERB change_VERB for_ADP your_A...,More_ADJ Try_VERB Yahoo_PROPN Finance_PROPN on...,1,0.759607,0.702148,0.272887,0


In [254]:
from collections import Counter

In [269]:
incorrect_df = pd.concat([incorrect_df, 
                          incorrect_df.title.str.findall(r'_([A-Z]+)').apply(Counter).apply(pd.Series).fillna(0).astype(int)], axis=1)

In [270]:
incorrect_df

,label,title,text,class,pred_0,pred_1,pred_2,class_pred,ADV,PART,...,NOUN,PROPN,PUNCT,NUM,DET,PRON,INTJ,CCONJ,SYM,SPACE
123,news,How_ADV to_PART Talk_VERB to_ADP Your_ADJ Teen...,"More_ADJ For_ADP parents_NOUN ,_PUNCT talking_...",1,0.746912,0.720410,0.267320,0,1,1,...,2,1,2,0,0,0,0,0,0,0
229,news,Genius_PROPN Bar_PROPN secrets_NOUN revealed_V...,One_NUM of_ADP the_DET biggest_ADJ things_NOUN...,1,0.598378,0.581579,0.554686,0,0,0,...,3,2,1,0,0,0,0,0,0,0
347,news,13_NUM Gifts_NOUN For_ADP People_NOUN Who_NOUN...,Love_NOUN is_VERB n't_ADV all_DET we_PRON need...,1,0.635953,0.620099,0.478591,0,0,0,...,4,0,0,1,0,0,0,0,0,0
391,news,Deer_PROPN Debacle_NOUN Exacerbates_VERB New_P...,Desc_PROPN,1,0.416454,0.626502,0.691686,2,0,0,...,1,5,0,0,0,0,0,0,0,0
490,news,Donald_PROPN Trump_PROPN Pays_VERB Respects_NO...,President_PROPN Donald_PROPN Trump_PROPN trave...,1,0.712625,0.705318,0.316700,0,0,0,...,1,6,0,0,0,0,0,0,0,0
497,news,These_DET trapdoor_NOUN waterslides_NOUN plung...,"David_PROPN Ibekwe_PROPN Apr._PROPN 12_NUM ,_P...",1,0.634602,0.630173,0.469868,0,0,0,...,3,0,1,0,2,1,0,0,0,0
694,news,How_ADV '_PUNCT Jurassic_PROPN World_PROPN '_P...,It_PRON is_VERB well_ADV -_PUNCT known_VERB at...,1,0.720751,0.715298,0.298594,0,1,1,...,4,2,2,0,0,0,0,0,0,0
726,news,10_NUM things_NOUN you_PRON need_VERB to_PART ...,"Jonathan_PROPN Garber_PROPN ,_PUNCT Business_P...",1,0.759789,0.700965,0.273889,0,0,1,...,2,0,0,1,1,1,0,0,0,0
754,news,Regis_PROPN Philbin_PROPN :_PUNCT Trump_PROPN ...,Daytime_ADJ talk_NOUN show_NOUN legend_NOUN Re...,1,0.780099,0.662219,0.292325,0,0,0,...,2,4,5,0,0,0,1,1,0,0
991,news,What_NOUN will_VERB change_VERB for_ADP your_A...,More_ADJ Try_VERB Yahoo_PROPN Finance_PROPN on...,1,0.759607,0.702148,0.272887,0,0,0,...,2,0,0,1,0,0,0,0,0,0


In [ ]:
train_df.div()

In [ ]:
sns.heatmap(pos_train_df.groupby('class').mean())